In [ ]:
from ultralytics import YOLO
import cv2
import time

# Load both models
fire_model = YOLO('fire.pt')
fire_model.names[0] = "fire"

smoke_model = YOLO('smoke_detection.pt')
smoke_model.names[0] = "smoke"

# Initialize video capture
cap = cv2.VideoCapture(0)

# Detection tracking (only fire & smoke)
detection_timers = {
    'fire': None,
    'smoke': None
}
detection_alerts = {
    'fire': False,
    'smoke': False
}

CONFIDENCE_THRESHOLD = 0.3
ALERT_DELAY_SECONDS = 2

while True:
    ret, frame = cap.read()
    if not ret:
        break

    resized_frame = cv2.resize(frame, (640, 480))
    annotated_frame = resized_frame.copy()  # start with raw frame

    # Run both models
    fire_results = fire_model(resized_frame)[0]
    smoke_results = smoke_model(resized_frame)[0]

    # Reset detection flags
    label_detected = {'fire': False, 'smoke': False}

    # Process and draw fire detections
    for box in fire_results.boxes:
        conf = float(box.conf)
        if conf > CONFIDENCE_THRESHOLD:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = "fire"
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(annotated_frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            label_detected[label] = True

    # Process and draw smoke detections
    for box in smoke_results.boxes:
        conf = float(box.conf)
        if conf > CONFIDENCE_THRESHOLD:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = "smoke"
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (255, 165, 0), 2)
            cv2.putText(annotated_frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 165, 0), 2)
            label_detected[label] = True

    # Time check for alerts
    current_time = time.time()

    for label in detection_timers:
        if label_detected[label]:
            if detection_timers[label] is None:
                detection_timers[label] = current_time
            elif current_time - detection_timers[label] >= ALERT_DELAY_SECONDS:
                detection_alerts[label] = True
        else:
            detection_timers[label] = None
            detection_alerts[label] = False

    # Display alerts
    y_offset = 50
    for label, alert in detection_alerts.items():
        if alert:
            cv2.putText(
                annotated_frame,
                f"ALERT: {label.upper()} DETECTED!",
                (50, y_offset),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 0, 255),
                2,
                cv2.LINE_AA
            )
            y_offset += 40
            print(f"{label.upper()} DETECTED")

    # Show the result
    cv2.imshow('Fire & Smoke Detection', annotated_frame)

    # Exit on ESC key
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 66.0ms
Speed: 1.8ms preprocess, 66.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 215.5ms
Speed: 1.9ms preprocess, 215.5ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 52.5ms
Speed: 2.0ms preprocess, 52.5ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 201.8ms
Speed: 1.1ms preprocess, 201.8ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 40.0ms
Speed: 1.1ms preprocess, 40.0ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 188.5ms
Speed: 1.1ms preprocess, 188.5ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 39.9ms
Speed: 1.2ms preprocess, 39.9ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 192.8ms
Speed: 0.8ms preprocess, 1